In [1]:
import pandas as pd
from hydrology_api import HydrologyApi, Measure, process_hydrology_data

ModuleNotFoundError: No module named 'ujson'

In [2]:
api = HydrologyApi(max_threads=5)

## Download Level Data

In [3]:
stations = api.get_stations_on_river('River Wear', Measure.LEVEL)
print(f"Loading data for {len(stations)} stations: {stations['label'].values}")
level_df = api.load(Measure.LEVEL, stations)
level_df = process_hydrology_data(level_df)
level_df.info()

Loading data for 5 stations: ['Chester Le Street' 'Witton Park' 'Sunderland Bridge' 'Stanhope'
 'Durham New Elvet Bridge']
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=e7d8bbb6-5bba-4057-9f49-a299482c3348-level-i-900-m-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=05784319-693a-4d75-b29e-32f01a99ee4f-level-i-900-m-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=ddedb4d9-b2be-47c1-998d-acbc0ffb124b-level-i-900-m-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=b29c481a-5012-40f5-bb0c-f9370be34975-level-i-900-m-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=ba3f8598-e654-430d-9bb8-e1652e6ff93d-level-i-900-m-qualified

/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)
/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)
/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)
/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)
/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on i

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 602263 entries, 2007-01-01 00:00:00 to 2024-03-05 13:30:00
Freq: 15min
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Chester Le Street        602263 non-null  float32
 1   Durham New Elvet Bridge  575122 non-null  float32
 2   Stanhope                 512911 non-null  float32
 3   Sunderland Bridge        600714 non-null  float32
 4   Witton Park              596463 non-null  float32
dtypes: float32(5)
memory usage: 16.1 MB


## Download Flow Data

In [4]:
stations = api.get_stations_on_river('River Wear', Measure.FLOW)
print(f"Loading data for {len(stations)} stations: {stations['label'].values}")
flow_df = api.load(Measure.FLOW, stations)
flow_df = process_hydrology_data(flow_df)
flow_df.info()

Loading data for 4 stations: ['Chester Le Street' 'Witton Park' 'Sunderland Bridge' 'Stanhope']


Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=e7d8bbb6-5bba-4057-9f49-a299482c3348-flow-i-900-m3s-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=05784319-693a-4d75-b29e-32f01a99ee4f-flow-i-900-m3s-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=ddedb4d9-b2be-47c1-998d-acbc0ffb124b-flow-i-900-m3s-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=b29c481a-5012-40f5-bb0c-f9370be34975-flow-i-900-m3s-qualified&mineq-date=2007-01-01


/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)
/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)
/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)
/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 592345 entries, 2007-01-01 00:00:00 to 2023-11-23 06:00:00
Freq: 15min
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Chester Le Street  592345 non-null  float32
 1   Stanhope           502991 non-null  float32
 2   Sunderland Bridge  590796 non-null  float32
 3   Witton Park        586545 non-null  float32
dtypes: float32(4)
memory usage: 13.6 MB


We don't have flow data at New Elvet unfortunatly

## Download Rainfall Data

In [5]:
rainfall_stations = api.get_stations_close_to_with_measure(54.66305556, -1.67611111, 15, Measure.RAINFALL, limit=10)
bad_stations = ['15202aee-c5fd-404d-9de9-7357174ad10c']
rainfall_stations = rainfall_stations[~rainfall_stations['notation'].isin(bad_stations)].head(5)
print(f"Using {len(rainfall_stations)} rainfall stations: {rainfall_stations['label'].values}")

rainfall_df = api.load(Measure.RAINFALL, rainfall_stations)
rainfall_df = process_hydrology_data(rainfall_df)
rainfall_df.info()

Using 5 rainfall stations: ['Evenwood Gate' 'Harpington Hill Farm' 'Copley' 'Tunstall'
 'Darlington Lingfield Way']
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 602263 entries, 2007-01-01 00:00:00 to 2024-03-05 13:30:00
Freq: 15min
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Copley                    587661 non-null  float32
 1   Darlington Lingfield Way  553917 non-null  float32
 2   Evenwood Gate             528775 non-null  float32
 3   Harpington Hill Farm      534478 non-null  float32
 4   Tunstall                  557350 non-null  float32
dtypes: float32(5)
memory usage: 16.1 MB


Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=bf61ce31-b20e-4593-85dc-a083133b12ce-rainfall-t-900-mm-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=1dabd12c-1d2e-4765-ae38-a4d5a121928d-rainfall-t-900-mm-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=bc34e640-d9ae-4362-8804-25d66ca66e4d-rainfall-t-900-mm-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=051f1b2a-6aca-4402-8956-5474ad39b12a-rainfall-t-900-mm-qualified&mineq-date=2007-01-01
Loading from cache: https://environment.data.gov.uk/hydrology/data/batch-readings/batch/?measure=a8773476-0fde-40c7-a66b-0901e528e8f2-rainfall-t-900-mm-qualified&mineq-date=2007-01-01


/home/ejex/code/river-level-analysis/model_training/hydrology_api/hydrology_api.py:98: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)


In [6]:
df = pd.merge(
    level_df.add_prefix('Level '),
    flow_df.add_prefix('Flow '),
    left_index=True,
    right_index=True,
    how='outer',
)
df = pd.merge(
    df,
    rainfall_df.add_prefix('Rainfall '),
    left_index=True,
    right_index=True,
    how='outer',
)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 602263 entries, 2007-01-01 00:00:00 to 2024-03-05 13:30:00
Freq: 15min
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Level Chester Le Street            602263 non-null  float32
 1   Level Durham New Elvet Bridge      575122 non-null  float32
 2   Level Stanhope                     512911 non-null  float32
 3   Level Sunderland Bridge            600714 non-null  float32
 4   Level Witton Park                  596463 non-null  float32
 5   Flow Chester Le Street             592345 non-null  float32
 6   Flow Stanhope                      502991 non-null  float32
 7   Flow Sunderland Bridge             590796 non-null  float32
 8   Flow Witton Park                   586545 non-null  float32
 9   Rainfall Copley                    587661 non-null  float32
 10  Rainfall Darlington Lingfield Way  553917 non-null  float3

In [7]:
# Fill na on flow and level by linear interpolation
flow_and_level_cols = [col for col in df.columns if col.startswith('Flow') or col.startswith('Level')]
df[flow_and_level_cols] = df[flow_and_level_cols].interpolate('time')

rainfall_cols = [col for col in df.columns if col.startswith('Rainfall')]
df[rainfall_cols] = df[rainfall_cols].fillna(0)

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 602263 entries, 2007-01-01 00:00:00 to 2024-03-05 13:30:00
Freq: 15min
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Level Chester Le Street            602263 non-null  float32
 1   Level Durham New Elvet Bridge      602263 non-null  float32
 2   Level Stanhope                     602263 non-null  float32
 3   Level Sunderland Bridge            602263 non-null  float32
 4   Level Witton Park                  602263 non-null  float32
 5   Flow Chester Le Street             602263 non-null  float32
 6   Flow Stanhope                      602263 non-null  float32
 7   Flow Sunderland Bridge             602263 non-null  float32
 8   Flow Witton Park                   602263 non-null  float32
 9   Rainfall Copley                    602263 non-null  float32
 10  Rainfall Darlington Lingfield Way  602263 non-null  float3

In [10]:
df.to_parquet('data.parquet')
df.tail()

station,Level Chester Le Street,Level Durham New Elvet Bridge,Level Stanhope,Level Sunderland Bridge,Level Witton Park,Flow Chester Le Street,Flow Stanhope,Flow Sunderland Bridge,Flow Witton Park,Rainfall Copley,Rainfall Darlington Lingfield Way,Rainfall Evenwood Gate,Rainfall Harpington Hill Farm,Rainfall Tunstall
dateTime,,,,,,,,,,,,,,
2024-03-05 12:30:00,0.670,0.563,0.856,0.704,0.838,23.4,5.36,16.5,11.7,0.0,0.0,0.0,0.0,0.0
2024-03-05 12:45:00,0.671,0.569,0.855,0.711,0.836,23.4,5.36,16.5,11.7,0.0,0.0,0.0,0.0,0.0
2024-03-05 13:00:00,0.672,0.578,0.848,0.717,0.826,23.4,5.36,16.5,11.7,0.0,0.0,0.0,0.0,0.0
2024-03-05 13:15:00,0.672,0.588,0.844,0.720,0.827,23.4,5.36,16.5,11.7,0.0,0.0,0.0,0.0,0.0
2024-03-05 13:30:00,0.674,0.600,0.841,0.724,0.825,23.4,5.36,16.5,11.7,0.0,0.0,0.0,0.0,0.0
